In [33]:
import matplotlib.pyplot as plt
import geopandas as gpd
import numpy as np
import pandas as pd


In [3]:
labels = gpd.read_file('attributes_20220817.json')

In [60]:
labels.head()

,attribute_id,label_type,street_edge_id,osm_street_id,neighborhood,severity,is_temporary,agree_count,disagree_count,notsure_count,geometry
0,57259626,CurbRamp,22481,394459713,Portage Bay,3.0,False,0,0,1,POINT (-122.32030 47.64312)
1,57259627,CurbRamp,12051,13521798,Portage Bay,1.0,False,2,0,0,POINT (-122.32189 47.64326)
2,57259629,CurbRamp,12051,13521798,Portage Bay,1.0,False,2,1,0,POINT (-122.32210 47.64312)
3,57259630,CurbRamp,22481,394459713,Portage Bay,1.0,False,1,0,0,POINT (-122.32017 47.64315)
4,57259632,CurbRamp,6732,6456858,Portage Bay,1.0,False,2,0,0,POINT (-122.32120 47.65138)


In [61]:
#select label type = 'curb ramp'
curbs = labels[labels['label_type'] == 'CurbRamp']

In [62]:
print(len(curbs))

32097


In [81]:
# Define the zoom level and corresponding filter thresholds
zoom_level = 4
zoom_thresholds = {
    0: 0.05,  # 1% of data points
    1: 0.1,   # 10% of data points
    2: 0.25,  # 25% of data points
    3: 0.5,   # 50% of data points
    4: 1.0    # All data points
}

# # Extract longitude and latitude from the 'geometry' column
# curbs['longitude'] = curbs['geometry'].x
# curbs['latitude'] = curbs['geometry'].y

# # Calculate the longitude and latitude intervals for dividing regions
# longitude_interval = 0.01
# latitude_interval = 0.01

# # Group the data points based on longitude and latitude intervals
# curbs['longitude_group'] = (curbs['longitude'] // longitude_interval) * longitude_interval
# curbs['latitude_group'] = (curbs['latitude'] // latitude_interval) * latitude_interval
# grouped_labels = curbs.groupby(['longitude_group', 'latitude_group'])

In [83]:
# # Define the minimum number of data points to keep in a region for zoom level 1
# min_data_points = 1

# # Sample data points per smaller region based on the filter threshold
# filtered_labels = grouped_labels.apply(lambda x: x.sample(frac=min(filter_threshold, min_data_points/len(x))))
# Determine the filter threshold based on the zoom level
filter_threshold = zoom_thresholds.get(zoom_level, 1.0)


# Calculate the number of data points to select from zoom level 1
if zoom_level > 0:
    prev_zoom_level = zoom_level - 1

    filtered_labels_prev_zoom = gpd.read_file(f'filtered_curbs_zoomin_{prev_zoom_level}.geojson')

    # filter_threshold_prev_zoom = zoom_thresholds.get(zoom_level-1, 1.0)
    # filtered_labels_prev_zoom = gpd.read_file('attributes_20220817.json')

    filter_threshold = filter_threshold - zoom_thresholds.get(prev_zoom_level, 1.0)
    print(filter_threshold)

    # disjoint_labels = curbs.difference(filtered_labels_prev_zoom)
    # Perform the attribute-based join
    merged = curbs.merge(filtered_labels_prev_zoom, on='attribute_id', how='left', indicator=True)

    # Filter the merged GeoDataFrame to keep only the records in curbs but not in filtered_labels_prev_zoom
    filtered_curbs = merged[merged['_merge'] == 'left_only']

    # Drop the _merge column from the resulting GeoDataFrame
    disjoint_labels = filtered_curbs.drop(columns='_merge')
    print(len(curbs))
    print(len(filtered_labels_prev_zoom))
    print(len(disjoint_labels))

    # Convert `cache_gdf` to a GeoDataFrame
    cache_gdf = gpd.GeoDataFrame(disjoint_labels, columns=filtered_labels_prev_zoom.columns)

    grouped_labels = cache_gdf.groupby('neighborhood')

    cache = cache_gdf.apply(lambda x: x.sample(frac=filter_threshold, random_state=14))

    # Concatenate the two GeoDataFrames
    filtered_labels = pd.concat([filtered_labels_prev_zoom, cache])

else:
    # Group the data points based on the neighborhood column
    grouped_labels = curbs.groupby('neighborhood')

    # Sample data points per smaller region based on the filter threshold
    filtered_labels = grouped_labels.apply(lambda x: x.sample(frac=filter_threshold, random_state=14))

# Reset the index of the filtered labels
filtered_labels = filtered_labels.reset_index(drop=True)

# Visualize the filtered data
# filtered_labels.plot()

0.5
32097
13631
18466


In [84]:
# print the length of filtered_labels
print(len(filtered_labels))

22864


In [85]:
# Check if all 'id' values from filtered_labels_prev_zoom exist in filtered_labels
is_subset = filtered_labels_prev_zoom['attribute_id'].isin(filtered_labels['attribute_id']).all()

if is_subset:
    print("filtered_labels contains all the data points from filtered_labels_prev_zoom")
    filtered_labels.to_file(f'filtered_curbs_zoomin_{zoom_level}.geojson', driver='GeoJSON')
else:
    print("filtered_labels does not contain all the data points from filtered_labels_prev_zoom")


filtered_labels contains all the data points from filtered_labels_prev_zoom


In [18]:
filtered_labels.explore()

ImportError: The 'folium', 'matplotlib' and 'mapclassify' packages are required for 'explore()'. You can install them using 'conda install -c conda-forge folium matplotlib mapclassify' or 'pip install folium matplotlib mapclassify'.